## Image Feature Comparison using PyTorch

This script demonstrates how to load, preprocess, and compare two images using a deep learning model in PyTorch. The main steps of the code are as follows:

1. **Image Loading and Preprocessing**:
    - Two images are loaded from the specified file paths using OpenCV (`cv2.imread`).
    - The images are resized to the required shape using `cv2.resize` and normalized by scaling the pixel values between 0 and 1.
    - The images are then converted into PyTorch tensors using `torch.from_numpy`, making them compatible with the neural network model.

2. **Feature Extraction**:
    - The preprocessed images are reshaped to include a batch dimension (i.e., `(1, 3, IMG_SHAPE, IMG_SHAPE)`), which is necessary for passing the images through the model.
    - The model takes each image as input and produces a feature vector that represents the image in a high-dimensional space.

3. **Distance Calculation**:
    - The pairwise distance between the feature vectors of the two images is computed using `torch.pairwise_distance`. This function calculates the Euclidean distance between the two feature vectors, indicating how similar or different the images are in the model's feature space.
    
The final output is the computed distance between the two images, which reflects their similarity according to the model's learned representations.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # Importing PyTorch libraries for neural networks
from torchsummary import summary  # For summarizing the model architecture
from torchvision import transforms  # For performing transformations on images
from torch import utils  # Utilities for PyTorch like DataLoader and Tensor manipulation

import cv2  # OpenCV for image processing tasks
import numpy as np  # NumPy for numerical operations
import os  # For file and directory operations
import random  # For generating random values

from tqdm.auto import trange  # TQDM for progress bars in loops

In [ ]:
# Defining constants for training configuration
EPOCHS = 10              # Number of training epochs
DATA_SIZE = 5000         # Total size of the training dataset
BATCH_SIZE = 100         # Batch size for training
VALID_DATA = 500         # Total size of the validation dataset
VALID_BATCH = 100        # Batch size for validation
IMG_SHAPE = 64           # Image dimensions (assumes square images of shape 64x64)


## Model Descriptions

### 1. **ConvBlock**

The `ConvBlock` class represents a modular unit used in the `OneShot` architecture. Each block consists of several convolutional layers followed by batch normalization and non-linear activation using LeakyReLU. The block structure allows for both downsampling and maintaining feature resolution, depending on whether a stride or pooling operation is applied. 

Key Features:
- **n_layers**: Defines the number of layers in the block.
- **filters**: Number of convolutional filters per layer.
- **kernel size**: Determines the receptive field of the convolution.
- **growth_factor**: Controls the scaling of filters across layers.
- **stride**: When `True`, a downsampling convolution is added, else max pooling is applied.
- **LeakyReLU activation**: A non-linear activation function applied after each convolution and normalization.

### 2. **OneShot Model**

The `OneShot` model is a deep neural network designed to perform one-shot learning. It takes pairs or triplets of images as input and learns to map them to a feature space where similar images are closer together, while dissimilar images are far apart. The architecture is built from a sequence of `ConvBlock` layers that progressively extract more abstract features from the input images.

Key Features:
- **n_blocks**: The number of `ConvBlock` units used in the network.
- **n_high_refine**: Specifies how many blocks use a higher refinement (more convolution layers) for feature extraction.
- **filters**: The base number of filters used for the initial convolutional layers, which grows with each block according to the `growth_factor`.
- **start_kernel**: Defines the kernel size for the first convolutional layer.
- **dropout**: A dropout layer is used to reduce overfitting.
- **AdaptiveAvgPool2d**: The network uses adaptive average pooling to reduce the final feature map size to (1,1) before flattening the features.
- **Fully Connected Layers**: After the convolutional layers, a fully connected (dense) layer maps the output to the final dimensionality, representing the learned feature vector for the input image.

### 3. **Triplet Loss**

The model is trained using a **Triplet Margin Loss**, which works by comparing anchor, positive, and negative samples:
- **Anchor**: The reference image.
- **Positive**: An image of the same class or label as the anchor.
- **Negative**: An image of a different class from the anchor.

The goal is to minimize the distance between the anchor and the positive image, while maximizing the distance between the anchor and the negative image in the feature space. The margin ensures that the negative sample is sufficiently far from the anchor.


In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, n_layers: int, filters: int, kernel: int = 3, growth_factor: float = 2.0, 
                 moment: float = 0.7, stride: bool = True, alpha: float = 0.03):
        super(ConvBlock, self).__init__()
        PADDING = (kernel - 1) // 2  # Padding based on the kernel size
        self.stride = stride

        # Batch normalization layers
        self.norm = nn.ModuleList([nn.BatchNorm1d(num_features=filters, momentum=moment) for _ in range(n_layers)])
        
        # Convolutional layers
        self.conv = nn.ModuleList([nn.Conv2d(filters, filters, kernel, padding=PADDING) for _ in range(n_layers - 1)])
        
        # Non-linear activation function
        self.nlin = nn.LeakyReLU(alpha)

        # Add final convolutional layer with stride or max pooling if applicable
        if stride:
            self.conv.append(nn.Conv2d(int(filters // growth_factor), filters, kernel, stride=2, padding=PADDING))
        else:
            self.conv.append(nn.Conv2d(int(filters // growth_factor), filters, kernel, padding=PADDING))
            self.pool = nn.MaxPool2d(2, 2)  # Max pooling layer

        self.conv.reverse()  # Reverse the convolution layers for the forward pass

    def forward(self, x):
        if not self.stride:
            x = self.pool(x)
        
        # Pass through convolutional and normalization layers with activation
        for conv, norm in zip(self.conv, self.norm):
            x = self.nlin(norm(conv(x)))
        return x

In [ ]:
class OneShot(nn.Module):
    def __init__(self, n_blocks: int = 7, n_high_refine: int = 3, n_conv_high_refine: int = 3, 
                 n_conv_end: int = 2, filters: int = 64, start_kernel: int = 5, kernel: int = 3, 
                 growth_factor: float = 2.0, alpha: float = 0.07, moment: float = 0.7, dense: int = 512, 
                 final: int = 100, drop: float = 0.2, stride: bool = True):
        super(OneShot, self).__init__()

        START_PADDING = (start_kernel - 1) // 2  # Padding for the starting convolution
        PADDING = (kernel - 1) // 2  # Padding for regular convolution layers

        # Initial convolutional layer with stride
        self.conv = nn.ModuleList([nn.Conv2d(3, filters, start_kernel, stride=2, padding=START_PADDING)])
        
        # Adding convolutional blocks
        for c in range(n_blocks):
            filters = int(filters * growth_factor)
            if c <= n_high_refine:
                # Add a high refinement ConvBlock
                self.conv.append(ConvBlock(n_conv_high_refine, filters, kernel, growth_factor, moment, stride, alpha))
            else:
                # Add an end refinement ConvBlock
                self.conv.append(ConvBlock(n_conv_end, filters, kernel, growth_factor, moment, stride, alpha))
        
        # Adaptive average pooling to reduce feature map size
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Flatten the output of the pooling layer
        self.flat = nn.Flatten()
        
        # Fully connected layers for classification
        self.lden = nn.Linear(int(filters * growth_factor), dense)
        self.lvec = nn.Linear(dense, final)
        
        # Activation and dropout
        self.nlin = nn.LeakyReLU(alpha)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        # Pass input through each convolutional block
        for conv in self.conv:
            x = conv(x)

        # Pool, flatten, and apply the fully connected layers with dropout
        x = self.flat(self.pool(x))
        x = self.drop(self.nlin(self.lden))
        return self.lvec(x)

In [ ]:
model = OneShot()
summary(model, (3, IMG_SHAPE, IMG_SHAPE))


In [ ]:
FOLDER = r"PATH"  # Path to the dataset folder
folders = os.listdir(FOLDER)  # List all subfolders (classes)
train = []  # List to store triplet (anchor, positive, negative) samples

for folder in folders:
    files = os.listdir(FOLDER + "/" + folder)  # List all files in the current folder

    for i in range(len(files) - 1):
        nfolder = random.choice(folders)  # Randomly select a different folder for the negative sample

        if nfolder != folder:
            path = r"PATH"  # Path for negative sample from another folder
            negative = cv2.imread(path)  # Load negative image from a different folder
        else:
            path = r"PATH"  # Path for negative sample (you may want to change this logic)
            negative = cv2.imread(path)  # Load negative image from the same folder

        # Load the anchor and positive samples from the current folder
        anchor = cv2.imread(FOLDER + "/" + folder + "/" + files[i])
        positive = cv2.imread(FOLDER + "/" + folder + "/" + files[i + 1])

        # Resize the images to the desired shape and normalize
        anchor = cv2.resize(anchor, (IMG_SHAPE, IMG_SHAPE), cv2.INTER_LINEAR) / 255.0
        positive = cv2.resize(positive, (IMG_SHAPE, IMG_SHAPE), cv2.INTER_LINEAR) / 255.0
        negative = cv2.resize(negative, (IMG_SHAPE, IMG_SHAPE), cv2.INTER_LINEAR) / 255.0

        # Append the triplet (anchor, positive, negative) to the training list
        train.append([anchor, positive, negative])

    # For folders with 4 or more files, create additional triplet samples
    if len(files) >= 4:
        for i in range(2):
            if nfolder != folder:
                fk = os.listdir(FOLDER + "/" + nfolder)[0]  # Select the first file from the negative folder
                negative = cv2.imread(FOLDER + "/" + nfolder + "/" + fk)
            else:
                path = r"PATH"  # Path for the negative sample
                negative = cv2.imread(path)

            # Load the anchor and positive samples from the current folder
            anchor = cv2.imread(FOLDER + "/" + folder + "/" + files[-2 + i])
            positive = cv2.imread(FOLDER + "/" + folder + "/" + files[0 + i])

            # Resize and normalize the images
            anchor = cv2.resize(anchor, (IMG_SHAPE, IMG_SHAPE), cv2.INTER_LINEAR) / 255.0
            positive = cv2.resize(positive, (IMG_SHAPE, IMG_SHAPE), cv2.INTER_LINEAR) / 255.0
            negative = cv2.resize(negative, (IMG_SHAPE, IMG_SHAPE), cv2.INTER_LINEAR) / 255.0

            # Append the triplet to the training list
            train.append([anchor, positive, negative])

# Convert the training list to a NumPy array
x = np.array(train, dtype=np.float32)


In [ ]:
# Convert NumPy arrays to PyTorch tensors for the training data (anchor, positive, negative)
xtrain1 = torch.from_numpy(x[:DATA_SIZE, 0].reshape(DATA_SIZE, 3, IMG_SHAPE, IMG_SHAPE))  # Anchor images
xtrain2 = torch.from_numpy(x[:DATA_SIZE, 1].reshape(DATA_SIZE, 3, IMG_SHAPE, IMG_SHAPE))  # Positive images
xtrain3 = torch.from_numpy(x[:DATA_SIZE, 2].reshape(DATA_SIZE, 3, IMG_SHAPE, IMG_SHAPE))  # Negative images
# ytrain = torch.from_numpy(y[:DATA_SIZE])  # Uncomment if you have labels (optional)

# Convert NumPy arrays to PyTorch tensors for the validation data (anchor, positive, negative)
xvalid1 = torch.from_numpy(x[DATA_SIZE:DATA_SIZE + VALID_DATA, 0].reshape(VALID_DATA, 3, IMG_SHAPE, IMG_SHAPE))  # Anchor images
xvalid2 = torch.from_numpy(x[DATA_SIZE:DATA_SIZE + VALID_DATA, 1].reshape(VALID_DATA, 3, IMG_SHAPE, IMG_SHAPE))  # Positive images
xvalid3 = torch.from_numpy(x[DATA_SIZE:DATA_SIZE + VALID_DATA, 2].reshape(VALID_DATA, 3, IMG_SHAPE, IMG_SHAPE))  # Negative images
# yvalid = torch.from_numpy(y[DATA_SIZE:DATA_SIZE + VALID_DATA])  # Uncomment if you have labels (optional)


In [ ]:
# Creating DataLoader objects for training data (anchor, positive, negative)
xtrain1 = utils.data.DataLoader(xtrain1, batch_size=BATCH_SIZE)  # DataLoader for anchor images in training
xtrain2 = utils.data.DataLoader(xtrain2, batch_size=BATCH_SIZE)  # DataLoader for positive images in training
xtrain3 = utils.data.DataLoader(xtrain3, batch_size=BATCH_SIZE)  # DataLoader for negative images in training
# ytrain = utils.data.DataLoader(ytrain, batch_size=BATCH_SIZE)  # Uncomment if you have labels

# Creating DataLoader objects for validation data (anchor, positive, negative)
xvalid1 = utils.data.DataLoader(xvalid1, batch_size=VALID_BATCH)  # DataLoader for anchor images in validation
xvalid2 = utils.data.DataLoader(xvalid2, batch_size=VALID_BATCH)  # DataLoader for positive images in validation
xvalid3 = utils.data.DataLoader(xvalid3, batch_size=VALID_BATCH)  # DataLoader for negative images in validation
# yvalid = utils.data.DataLoader(yvalid, batch_size=VALID_BATCH)  # Uncomment if you have labels


## Training Process

### 1. **Data Preparation**
The training process begins with the creation of **triplet data**:
- **Anchor**: The reference image from a specific class.
- **Positive**: An image of the same class as the anchor.
- **Negative**: An image from a different class than the anchor.

These triplets are loaded into PyTorch **DataLoader** objects to ensure efficient batching and iteration through the dataset.

### 2. **Loss Function**
The model is trained using **Triplet Margin Loss** with a pairwise distance function (`nn.PairwiseDistance()`). The objective is to minimize the distance between the anchor and the positive image while ensuring that the negative image is sufficiently distant. The margin is set to 10 in this case, meaning the negative sample should be at least this distance away from the anchor.

### 3. **Optimizer and Learning Rate**
The **Adam optimizer** is used for model optimization. The learning rate is initialized at `1e-4`, and **learning rate decay** is applied dynamically after each epoch using the following formula:

learning_rate = learning_rate / (1 + epoch * decay)


Where `decay = 0.9`. This progressively reduces the learning rate as training continues, helping the model converge smoothly.

### 4. **Training Loop**
For each epoch, the following operations are performed:

- **Forward Pass**:
    - For each batch, the triplet (anchor, positive, negative) is passed through the model.
    - The model generates a feature vector for each image.
    
- **Loss Calculation**:
    - The **Triplet Loss** is computed using the predicted feature vectors of the anchor, positive, and negative images.
    - The total loss for the epoch is accumulated over all batches.

- **Backpropagation and Optimization**:
    - The model’s gradients are reset with `optimizer.zero_grad()`.
    - The computed loss is backpropagated using `loss.backward()`.
    - The optimizer updates the model’s weights using `optimizer.step()`.

### 5. **Validation**
After each training epoch, the model is evaluated on a separate validation set. The same triplet structure (anchor, positive, negative) is used to compute validation loss, providing a measure of the model's generalization to unseen data.

### 6. **Performance Monitoring**
For each epoch, the training and validation loss are printed to track the model's performance over time. This helps in ensuring the model is improving and not overfitting to the training data.


In [ ]:
# Calculate the number of steps per epoch based on training and validation batches
steps = len(xtrain1)  # Number of steps (batches) in training data
vstep = len(xvalid1)  # Number of steps (batches) in validation data

# Define the triplet loss function with pairwise distance
criterion = nn.TripletMarginWithDistanceLoss(distance_function=nn.PairwiseDistance(), margin=10)

# Set initial learning rate and decay factor for learning rate scheduling
learning_rate = 1e-4
decay = 0.9

# Training loop
for epoch in trange(EPOCHS):  # Iterate through epochs
    lss = 0  # Initialize training loss for the epoch
    vls = 0  # Initialize validation loss for the epoch
    
    # Decay learning rate over time
    learning_rate = learning_rate / (1 + epoch * decay)
    
    # Optimizer with the updated learning rate
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop over batches
    for step, (xtr1, xtr2, xtr3) in enumerate(zip(xtrain1, xtrain2, xtrain3)):
        a = model(xtr1)  # Anchor output
        p = model(xtr2)  # Positive output
        n = model(xtr3)  # Negative output
        
        # Compute triplet loss
        loss = criterion(a, p, n)
        lss += loss  # Accumulate training loss
        
        # Zero the gradients, backpropagate, and update the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation loop over batches
    for step, (xval1, xval2, xval3) in enumerate(zip(xvalid1, xvalid2, xvalid3)):
        a = model(xval1)  # Anchor output for validation
        p = model(xval2)  # Positive output for validation
        n = model(xval3)  # Negative output for validation
        
        # Compute validation loss
        valloss = criterion(a, p, n)
        vls += valloss  # Accumulate validation loss
    
    # Print average training and validation loss for the current epoch
    print(f'Epoch: {epoch+1}/{EPOCHS} || Loss: {lss/steps:.4f} || Validation Loss: {vls/vstep:.4f}')


In [ ]:
# Load two images from the provided paths and preprocess them
img1 = r"PATH"  # Path to the first image
img2 = r"PATH"  # Path to the second image

# Read, resize, and normalize image1, then convert it to a PyTorch tensor
img1 = torch.from_numpy(np.array(cv2.resize(cv2.imread(img1), (IMG_SHAPE, IMG_SHAPE), cv2.INTER_LINEAR) / 255.0, dtype=np.float32))

# Read, resize, and normalize image2, then convert it to a PyTorch tensor
img2 = torch.from_numpy(np.array(cv2.resize(cv2.imread(img2), (IMG_SHAPE, IMG_SHAPE), cv2.INTER_LINEAR) / 255.0, dtype=np.float32))

# Reshape the images to the format expected by the model: (batch_size, channels, height, width)
pred1 = model(img1.reshape(1, 3, IMG_SHAPE, IMG_SHAPE))  # Predict features for image1
pred2 = model(img2.reshape(1, 3, IMG_SHAPE, IMG_SHAPE))  # Predict features for image2

# Calculate the pairwise distance between the two predicted feature vectors
distance = torch.pairwise_distance(pred1, pred2)

# Output the calculated distance
print(distance)
